In [1]:
import pickle
from ppp_prediction import transtab
import pandas as pd

path_dir = "result/dl/transtab/data/proteomics_pretrain"
total_save_dict = pickle.load(open(f"{path_dir}/total.pkl", "rb"))

allset = total_save_dict["allset"]
trainset = total_save_dict["trainset"]
valset = total_save_dict["valset"]
testset = total_save_dict["testset"]
cat_cols = total_save_dict["cat_cols"]
num_cols = total_save_dict["num_cols"]
bin_cols = total_save_dict["bin_cols"]

## build tokenizer

In [5]:
testset[0]

,C3,KLK7,GCHFR,NHLRC3,APOD,GAPDH,TP53I3,CPA4,ANXA2,GRSF1,...,EGFR,TGFBR3,CRTAC1,IGFBP7,SELE,VWF,NOTCH3,CNTN1,ENG,ICAM2
15198,0.246386,0.392439,0.422629,0.390376,0.186254,0.716298,0.611781,0.634428,0.391027,0.404884,...,0.534404,0.539900,0.484547,0.258455,0.569050,0.371777,0.539122,0.684294,0.329530,0.496316
35100,0.224977,0.383422,0.337074,0.437169,0.227823,0.694511,0.561354,0.635350,0.378167,0.350780,...,0.523701,0.539058,0.739998,0.273154,0.589236,0.529724,0.441691,0.642650,0.351770,0.418775
42068,0.146543,0.413603,0.407486,0.390111,0.239150,0.671613,0.617724,0.667863,0.328016,0.294891,...,0.539120,0.654190,0.727452,0.267658,0.470695,0.465224,0.524944,0.671121,0.374023,0.494131
16859,0.189689,0.347109,0.356901,0.457914,0.122809,0.593918,0.532941,0.624265,0.378167,0.244164,...,0.579374,0.596961,0.691391,0.258063,0.506645,0.472061,0.347189,0.670706,0.347567,0.371712
31040,0.206070,0.303563,0.313697,0.480382,0.276675,0.708501,0.536795,0.609719,0.374876,0.373608,...,0.537106,0.553449,0.673058,0.307715,0.457352,0.511037,0.389063,0.683750,0.377309,0.257669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14107,0.224977,0.383422,0.337074,0.437169,0.227823,0.694511,0.561354,0.635350,0.378167,0.350780,...,0.558429,0.549014,0.686394,0.493021,0.628338,0.538094,0.523766,0.660475,0.350839,0.343446
21136,0.263436,0.413487,0.338355,0.539016,0.224030,0.741229,0.553085,0.696076,0.395797,0.269579,...,0.516661,0.516600,0.635209,0.364612,0.606724,0.366479,0.377790,0.639901,0.374080,0.314928
30453,0.256631,0.401165,0.290320,0.359248,0.165589,0.654123,0.529941,0.678682,0.498892,0.367609,...,0.426126,0.473687,0.696126,0.231741,0.296297,0.386632,0.321364,0.592886,0.230461,0.263944
3942,0.198270,0.363439,0.288320,0.508550,0.221158,0.604816,0.450000,0.643523,0.378033,0.335829,...,0.471312,0.471348,0.603909,0.265504,0.603901,0.424027,0.338448,0.659228,0.333380,0.406421


In [34]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    processors,
)
from transformers import BertTokenizer, BertTokenizerFast, PreTrainedTokenizerFast

PAD = "[PAD]"

CLS = "[CLS]"
SEP = "[SEP]"
MASK = "[MASK]"
UNK = "[UNK]"
lowercase = True

token_list = trainset[0].columns.tolist()

tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.BertNormalizer(
    lowercase=lowercase,
    clean_text=False,
    handle_chinese_chars=False,
    strip_accents=False,
)
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# tokenizer.decoder = decoders.
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

tokenizer.post_process = processors.BertProcessing(sep=("[SEP]", 2), cls=("[CLS]", 1))
trainer = trainers.WordLevelTrainer(
    vocab_size=len(token_list) + 100,
    special_tokens=[PAD, CLS, SEP, MASK, UNK],
    show_progress=True,
)
print(tokenizer.get_vocab_size())
tokenizer.train_from_iterator(token_list, trainer=trainer)
print(tokenizer.get_vocab_size())
# tokenizer.get_vocab()
# BertTokenizerFast
bert_tokenizer = BertTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token=UNK,
    pad_token=PAD,
    cls_token=CLS,
    sep_token=SEP,
    mask_token=MASK,
    lowercase=lowercase,
)
bert_tokenizer.model_max_length = len(token_list)

0
2916


In [36]:
recode_list = tokenizer.decode(tokenizer.encode(" ".join(token_list)).ids).split(" ")
recode_list

['c3',
 'klk7',
 'gchfr',
 'nhlrc3',
 'apod',
 'gapdh',
 'tp53i3',
 'cpa4',
 'anxa2',
 'grsf1',
 'il25',
 'hmmr',
 'mrpl52',
 'paip2b',
 'thap12',
 'fos',
 'fgf9',
 'pithd1',
 'thsd1',
 'ptges2',
 'defb103a_defb103b',
 'atp1b4',
 'cyb5a',
 'unc79',
 'slc34a3',
 'tagln3',
 'slirp',
 'clasp1',
 'psmc3',
 'kir3dl2',
 'bex3',
 'pfdn4',
 'bcl7a',
 'smc3',
 'slc28a1',
 'cdc123',
 'gja8',
 'nmrk2',
 'gata3',
 'cplx2',
 'rasgrf1',
 'fgf7',
 'ankra2',
 'rbm25',
 'lyzl2',
 'cdk1',
 'creb3',
 'crebzf',
 'iglon5',
 'shc1',
 'zp4',
 'tmod4',
 'cep152',
 'myh7b',
 'cep350',
 'cdc25a',
 'trim26',
 'maneal',
 'mucl3',
 'gimap8',
 'cyth3',
 'pdxdc1',
 'clint1',
 'mapre3',
 'evi2b',
 'stau1',
 'pcna',
 'dnaja1',
 'jmjd1c',
 'gage2a',
 'gad1',
 'izumo1',
 'pdcl2',
 'pde1c',
 'stoml2',
 'bsnd',
 'mapk13',
 'pdia2',
 'btla',
 'mllt1',
 'tprkb',
 'arhgap5',
 'btnl10',
 'phldb2',
 'pdia5',
 'atf4',
 'prame',
 'top1mt',
 'khdc3l',
 'dcun1d2',
 'il3',
 'dclre1c',
 'ercc1',
 'dcdc2c',
 'vcpkmt',
 'spring1',
 'm

In [37]:
pd.DataFrame(
    {"token_list": [i.lower() for i in token_list], "recode_list": recode_list[:2911]}
).to_csv("test.csv")

In [38]:
len(token_list)

2911

In [39]:
bert_tokenizer.save_pretrained(f"./transtab/tokenizer")

('./transtab/tokenizer/tokenizer_config.json',
 './transtab/tokenizer/special_tokens_map.json',
 './transtab/tokenizer/vocab.json',
 './transtab/tokenizer/added_tokens.json',
 './transtab/tokenizer/tokenizer.json')

In [ ]:
bert_tokenizer()

## pretrain

In [15]:
model, collate_fn = transtab.build_contrastive_learner(
    categorical_columns=cat_cols,
    numerical_columns=num_cols,
    binary_columns=bin_cols,
    supervised=False,  # if take supervised CL
    num_partition=4,  # num of column partitions for pos/neg sampling
    overlap_ratio=0.1,  # specify the overlap ratio of column partitions during the CL
)

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [17]:
# training_arguments = {
#     "num_epoch": 1,
#     "batch_size": 32,
#     "lr": 1e-4,
#     "eval_metric": "val_loss",
#     "eval_less_is_better": True,
#     "output_dir": "./checkpoint",  # save the pretrained model
# }
# trainer = transtab.Trainer(
#     model, trainset, valset, collate_fn=collate_fn, **training_arguments
# )


# for x, y in trainer.trainloader_list[0]:
#     break

In [18]:
# start contrastive pretraining training
training_arguments = {
    "num_epoch": 1,
    "batch_size": 32,
    "lr": 1e-4,
    "eval_metric": "val_loss",
    "eval_less_is_better": True,
    "output_dir": "./checkpoint",  # save the pretrained model
}

In [19]:
transtab.train(model, valset, valset, collate_fn=collate_fn, **training_arguments)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

2024-04-19 17:59:07.697 | INFO     | ppp_prediction.transtab.trainer:train:137 - load best at last from ./checkpoint
2024-04-19 17:59:07.703 | INFO     | ppp_prediction.transtab.trainer:save_model:248 - saving model checkpoint to ./checkpoint
2024-04-19 17:59:07.730 | INFO     | ppp_prediction.transtab.trainer:train:142 - training complete, cost 39.7 secs.


epoch: 0, test val_loss: 5.979982
epoch: 0, train loss: 17.2189, lr: 0.000100, spent: 39.7 secs


## transfer learning

In [4]:
from torch.utils.data import WeightedRandomSampler
from sklearn.utils import class_weight
import numpy as np


def createWeightedSampler(labels, class_num=2):

    if isinstance(labels, pd.Series):
        labels = labels.values

    class_weights = dict(
        enumerate(
            class_weight.compute_class_weight(
                "balanced",
                classes=np.arange(class_num),
                y=labels,
            )
        )
    )
    print(class_weights)
    train_class_weights = [class_weights[i] for i in labels]
    sampler = WeightedRandomSampler(
        train_class_weights, len(train_class_weights), replacement=True
    )
    return sampler


sampler = createWeightedSampler(trainset[1])

{0: 0.532724505327245, 1: 8.13953488372093}


In [2]:
model = transtab.build_classifier(checkpoint="./checkpoint")
model.update({"cat": cat_cols, "num": num_cols, "bin": bin_cols})

2024-04-19 17:53:14.650 | WARNING  | ppp_prediction.transtab.modeling_transtab:_check_column_overlap:258 - No cat/num/bin cols specified, will take ALL columns as categorical! Ignore this warning if you specify the `checkpoint` to load the model.
/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
2024-04-19 17:53:14.864 | INFO     | ppp_prediction.transtab.modeling_transtab:load:779 - missing keys: ['clf.fc.weight', 'clf.fc.bias', 'clf.norm.weight', 'clf.norm.bias']
2024-04-19 17:53:14.865 | INFO     | ppp_prediction.transtab.modeling_transtab:load:780 - unexpected keys: ['projection_head.dense.weight']
2024-04-19 17:53:14.866 | INFO     | ppp_prediction.transtab.modeling_tr

In [3]:
training_arguments = {
    "num_epoch": 50,
    "eval_metric": "val_loss",
    "eval_less_is_better": True,
    "output_dir": "./finetune",
}
transtab.train(model, trainset, valset, **training_arguments)

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
feature_extractor = TransTabFeatureExtractor(
    cat_cols,
    num_cols,
    bin_cols,
    BertTokenDir="proteomics_bert_tokenizer",
)

load tokenizer from proteomics_bert_tokenizer


In [17]:
from transtab.modeling_transtab import TransTabFeatureProcessor


feature_processor = TransTabFeatureProcessor(
    vocab_size=feature_extractor.vocab_size,
    pad_token_id=feature_extractor.pad_token_id,
    hidden_dim=128,
    hidden_dropout_prob=0.1,
    device="cpu",
)

In [136]:
from transformers import BertTokenizer, BertTokenizerFast, PreTrainedTokenizerFast

toknizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
toknizer(["txn", "[MASK]"])

{'input_ids': [[101, 19067, 2078, 102], [101, 103, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1]]}

In [109]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    processors,
)
from transformers import BertTokenizer, BertTokenizerFast, PreTrainedTokenizerFast


tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False)
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# tokenizer.decoder = decoders.
tokenizer.post_process = processors.BertProcessing(sep=("[SEP]", 2), cls=("[CLS]", 1))
trainer = trainers.WordLevelTrainer(
    vocab_size=3000,
    special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"],
    show_progress=True,
)
tokenizer.train_from_iterator([trainset[0].columns.tolist()], trainer=trainer)

In [100]:
" ".join(trainset[0].columns.tolist())

'C3 KLK7 GCHFR NHLRC3 APOD GAPDH TP53I3 CPA4 ANXA2 GRSF1 IL25 HMMR MRPL52 PAIP2B THAP12 FOS FGF9 PITHD1 THSD1 PTGES2 DEFB103A_DEFB103B ATP1B4 CYB5A UNC79 SLC34A3 TAGLN3 SLIRP CLASP1 PSMC3 KIR3DL2 BEX3 PFDN4 BCL7A SMC3 SLC28A1 CDC123 GJA8 NMRK2 GATA3 CPLX2 RASGRF1 FGF7 ANKRA2 RBM25 LYZL2 CDK1 CREB3 CREBZF IGLON5 SHC1 ZP4 TMOD4 CEP152 MYH7B CEP350 CDC25A TRIM26 MANEAL MUCL3 GIMAP8 CYTH3 PDXDC1 CLINT1 MAPRE3 EVI2B STAU1 PCNA DNAJA1 JMJD1C GAGE2A GAD1 IZUMO1 PDCL2 PDE1C STOML2 BSND MAPK13 PDIA2 BTLA MLLT1 TPRKB ARHGAP5 BTNL10 PHLDB2 PDIA5 ATF4 PRAME TOP1MT KHDC3L DCUN1D2 IL3 DCLRE1C ERCC1 DCDC2C VCPKMT SPRING1 MORN4 ESPL1 H2AP MORF4L2 SSH3 VWA5A PBK REST SHD TXNL1 TPM3 NEB ATP1B2 CEP112 SART1 ATP6V1G2 ATP2B4 SAT1 ATP1B1 NECAP2 ATP5F1D ATP1B3 ARNTL ARL2BP SCGB2A2 GAMT ASS1 NFYA GASK1A MANSC4 HMGCS1 MMUT CBX2 BRD3 BRDT MAP1LC3B2 CASQ2 HIP1 GSTM4 GUK1 CALY C1GALT1C1 TEF CACNA1H HADH MEGF11 MED21 THRAP3 SPINK8 NAA10 MRPL24 GBP6 MYOM2 B3GAT3 GCLM MYL1 HSD17B3 MYH4 TMED4 TMED10 SKIV2L SLC12A2 SL

In [112]:
tokenizer.encode(" ".join(trainset[0].columns.tolist()))

Encoding(num_tokens=1, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [113]:
from transformers import PreTrainedTokenizerFast, BertTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # 或者从文件加载
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [121]:
bert_tokenizer = BertTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    do_lower_case=False,
)

In [122]:
wrapped_tokenizer(trainset[0].columns.tolist())

{'input_ids': [[316], [1519], [1132], [1858], [156], [1116], [2746], [655], [136], [1220], [1398], [1277], [1756], [1959], [2660], [1076], [1043], [2048], [2667], [2177], [795], [208], [751], [2819], [2467], [2609], [2475], [578], [2165], [1501], [258], [2022], [253], [2488], [2466], [479], [1166], [1870], [1122], [659], [2238], [1042], [129], [2244], [1645], [499], [672], [673], [1350], [2431], [2914], [2697], [527], [1786], [532], [480], [2775], [1657], [1778], [1160], [754], [2010], [598], [1671], [971], [2568], [1984], [823], [1475], [1108], [1104], [1471], [1993], [1994], [2576], [291], [1668], [2002], [296], [1724], [2758], [175], [300], [2035], [2005], [201], [2120], [2741], [1487], [778], [1402], [770], [958], [769], [2841], [2543], [1745], [968], [1239], [1744], [2557], [2870], [1972], [2264], [2432], [2802], [2752], [1829], [206], [526], [2335], [217], [209], [2336], [205], [1830], [210], [207], [186], [183], [2349], [1115], [199], [1854], [1120], [1660], [1276], [1736], [374

In [123]:
wrapped_tokenizer.decode(
    [i[0] for i in wrapped_tokenizer(trainset[0].columns.tolist())["input_ids"]]
)

'C3 KLK7 GCHFR NHLRC3 APOD GAPDH TP53I3 CPA4 ANXA2 GRSF1 IL25 HMMR MRPL52 PAIP2B THAP12 FOS FGF9 PITHD1 THSD1 PTGES2 DEFB103A_DEFB103B ATP1B4 CYB5A UNC79 SLC34A3 TAGLN3 SLIRP CLASP1 PSMC3 KIR3DL2 BEX3 PFDN4 BCL7A SMC3 SLC28A1 CDC123 GJA8 NMRK2 GATA3 CPLX2 RASGRF1 FGF7 ANKRA2 RBM25 LYZL2 CDK1 CREB3 CREBZF IGLON5 SHC1 ZP4 TMOD4 CEP152 MYH7B CEP350 CDC25A TRIM26 MANEAL MUCL3 GIMAP8 CYTH3 PDXDC1 CLINT1 MAPRE3 EVI2B STAU1 PCNA DNAJA1 JMJD1C GAGE2A GAD1 IZUMO1 PDCL2 PDE1C STOML2 BSND MAPK13 PDIA2 BTLA MLLT1 TPRKB ARHGAP5 BTNL10 PHLDB2 PDIA5 ATF4 PRAME TOP1MT KHDC3L DCUN1D2 IL3 DCLRE1C ERCC1 DCDC2C VCPKMT SPRING1 MORN4 ESPL1 H2AP MORF4L2 SSH3 VWA5A PBK REST SHD TXNL1 TPM3 NEB ATP1B2 CEP112 SART1 ATP6V1G2 ATP2B4 SAT1 ATP1B1 NECAP2 ATP5F1D ATP1B3 ARNTL ARL2BP SCGB2A2 GAMT ASS1 NFYA GASK1A MANSC4 HMGCS1 MMUT CBX2 BRD3 BRDT MAP1LC3B2 CASQ2 HIP1 GSTM4 GUK1 CALY C1GALT1C1 TEF CACNA1H HADH MEGF11 MED21 THRAP3 SPINK8 NAA10 MRPL24 GBP6 MYOM2 B3GAT3 GCLM MYL1 HSD17B3 MYH4 TMED4 TMED10 SKIV2L SLC12A2 SL

In [124]:
[i[0] for i in wrapped_tokenizer(trainset[0].columns.tolist())["input_ids"]]

[316,
 1519,
 1132,
 1858,
 156,
 1116,
 2746,
 655,
 136,
 1220,
 1398,
 1277,
 1756,
 1959,
 2660,
 1076,
 1043,
 2048,
 2667,
 2177,
 795,
 208,
 751,
 2819,
 2467,
 2609,
 2475,
 578,
 2165,
 1501,
 258,
 2022,
 253,
 2488,
 2466,
 479,
 1166,
 1870,
 1122,
 659,
 2238,
 1042,
 129,
 2244,
 1645,
 499,
 672,
 673,
 1350,
 2431,
 2914,
 2697,
 527,
 1786,
 532,
 480,
 2775,
 1657,
 1778,
 1160,
 754,
 2010,
 598,
 1671,
 971,
 2568,
 1984,
 823,
 1475,
 1108,
 1104,
 1471,
 1993,
 1994,
 2576,
 291,
 1668,
 2002,
 296,
 1724,
 2758,
 175,
 300,
 2035,
 2005,
 201,
 2120,
 2741,
 1487,
 778,
 1402,
 770,
 958,
 769,
 2841,
 2543,
 1745,
 968,
 1239,
 1744,
 2557,
 2870,
 1972,
 2264,
 2432,
 2802,
 2752,
 1829,
 206,
 526,
 2335,
 217,
 209,
 2336,
 205,
 1830,
 210,
 207,
 186,
 183,
 2349,
 1115,
 199,
 1854,
 1120,
 1660,
 1276,
 1736,
 374,
 285,
 286,
 1662,
 368,
 1266,
 1226,
 1233,
 350,
 302,
 2636,
 339,
 1240,
 1695,
 1693,
 2666,
 2534,
 1801,
 1753,
 1128,
 1797,
 229,
 

In [125]:
bert_tokenizer(trainset[0].columns.tolist())

{'input_ids': [[316], [1519], [1132], [1858], [156], [1116], [2746], [655], [136], [1220], [1398], [1277], [1756], [1959], [2660], [1076], [1043], [2048], [2667], [2177], [795], [208], [751], [2819], [2467], [2609], [2475], [578], [2165], [1501], [258], [2022], [253], [2488], [2466], [479], [1166], [1870], [1122], [659], [2238], [1042], [129], [2244], [1645], [499], [672], [673], [1350], [2431], [2914], [2697], [527], [1786], [532], [480], [2775], [1657], [1778], [1160], [754], [2010], [598], [1671], [971], [2568], [1984], [823], [1475], [1108], [1104], [1471], [1993], [1994], [2576], [291], [1668], [2002], [296], [1724], [2758], [175], [300], [2035], [2005], [201], [2120], [2741], [1487], [778], [1402], [770], [958], [769], [2841], [2543], [1745], [968], [1239], [1744], [2557], [2870], [1972], [2264], [2432], [2802], [2752], [1829], [206], [526], [2335], [217], [209], [2336], [205], [1830], [210], [207], [186], [183], [2349], [1115], [199], [1854], [1120], [1660], [1276], [1736], [374

In [73]:
tokenizer.save("test.json")

In [65]:
trainset[0].columns

Index(['C3', 'KLK7', 'GCHFR', 'NHLRC3', 'APOD', 'GAPDH', 'TP53I3', 'CPA4',
       'ANXA2', 'GRSF1',
       ...
       'EGFR', 'TGFBR3', 'CRTAC1', 'IGFBP7', 'SELE', 'VWF', 'NOTCH3', 'CNTN1',
       'ENG', 'ICAM2'],
      dtype='object', length=2911)

In [50]:
from transformers import BertTokenizer, BertTokenizerFast,PreTrainedTokenizerFast

toknizer = BertTokenizerFast.from_pretrained("transtab/tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [52]:
toknizer._tokenizer.

In [42]:
tokenizer.train_from_iterator([trainset[0].columns.tolist()], trainer=trainer)

In [44]:
from tokenizers import BertWordPieceTokenizer

In [37]:
toknizer.add_tokens(trainset[0].columns.tolist())
toknizer.vocab

{'IFNAR1': 1065,
 'BAG6': 1803,
 'ISLR2': 1464,
 'KIRREL1': 335,
 'IGDCC3': 321,
 'ASPN': 1291,
 'DAND5': 461,
 'PPIB': 2472,
 'CDON': 2483,
 'SRPK2': 2262,
 'RHOC': 1863,
 'THRAP3': 148,
 'IL17RB': 1862,
 'GSTA1': 2176,
 'ARID4B': 1658,
 'TANK': 1761,
 'SSC4D': 2183,
 'NXPE4': 184,
 'GABRA4': 273,
 'STAT2': 457,
 'PTPRC': 1363,
 'GART': 1265,
 'PDIA2': 82,
 'LYPD8': 1554,
 'CC2D1A': 1960,
 'TIA1': 1480,
 'NTRK3': 2789,
 'PER3': 473,
 'CELA3A': 2813,
 'SDCCAG8': 972,
 'MAPKAPK2': 797,
 'INSL3': 1148,
 'DAPP1': 2253,
 'DUSP3': 1687,
 'CXCL5': 2132,
 'IGF1R': 2029,
 'DDHD2': 1221,
 'UBE2L6': 1285,
 'LTBP2': 2885,
 'SPRR1B': 834,
 'INSL5': 1067,
 'PPP3R1': 1956,
 'SLC9A3R2': 1076,
 'RABGAP1L': 1735,
 'PFDN2': 1698,
 'RNASET2': 2700,
 'SERPINB8': 2832,
 'ENPP5': 2640,
 'CARHSP1': 1644,
 'FGL1': 1411,
 'ADGRE2': 2637,
 'PALM': 885,
 'TPPP2': 817,
 'MITD1': 1926,
 'CLEC12A': 390,
 'IL4': 1792,
 'PSMA1': 1586,
 'EEF1D': 1293,
 'CIRBP': 1036,
 'CLEC4M': 1268,
 'ZHX2': 1117,
 'CACYBP': 689,
 'C

In [ ]:
toknizer.p

In [38]:
toknizer(trainset[0].columns.tolist())

{'input_ids': [[101, 5, 102], [101, 6, 102], [101, 7, 102], [101, 8, 102], [101, 9, 102], [101, 10, 102], [101, 11, 102], [101, 12, 102], [101, 13, 102], [101, 14, 102], [101, 15, 102], [101, 16, 102], [101, 17, 102], [101, 18, 102], [101, 19, 102], [101, 20, 102], [101, 21, 102], [101, 22, 102], [101, 23, 102], [101, 24, 102], [101, 25, 102], [101, 26, 102], [101, 27, 102], [101, 28, 102], [101, 29, 102], [101, 30, 102], [101, 31, 102], [101, 32, 102], [101, 33, 102], [101, 34, 102], [101, 35, 102], [101, 36, 102], [101, 37, 102], [101, 38, 102], [101, 39, 102], [101, 40, 102], [101, 41, 102], [101, 42, 102], [101, 43, 102], [101, 44, 102], [101, 45, 102], [101, 46, 102], [101, 47, 102], [101, 48, 102], [101, 49, 102], [101, 50, 102], [101, 51, 102], [101, 52, 102], [101, 53, 102], [101, 54, 102], [101, 55, 102], [101, 56, 102], [101, 57, 102], [101, 58, 102], [101, 59, 102], [101, 60, 102], [101, 61, 102], [101, 62, 102], [101, 63, 102], [101, 64, 102], [101, 65, 102], [101, 66, 102]

In [2]:
model, collate_fn = transtab.build_contrastive_learner(
    categorical_columns=cat_cols,
    numerical_columns=num_cols,
    binary_columns=bin_cols,
    supervised=True,  # if take supervised CL
    num_partition=4,  # num of column partitions for pos/neg sampling
    overlap_ratio=0.1,  # specify the overlap ratio of column partitions during the CL
)

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [46]:
# start contrastive pretraining training
training_arguments = {
    "num_epoch": 50,
    "batch_size": 8,
    "lr": 1e-4,
    "eval_metric": "val_loss",
    "eval_less_is_better": True,
    "output_dir": "./checkpoint",  # save the pretrained model
}

In [10]:
# pass the collate function to the train function
transtab.train(model, trainset, valset, collate_fn=collate_fn, **training_arguments)

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [47]:
trainer = transtab.Trainer(
    model, trainset, valset, collate_fn=collate_fn, **training_arguments
)

In [48]:
%%time 
for x, y in trainer.trainloader_list[0]:
    break

CPU times: user 85 ms, sys: 6.27 ms, total: 91.2 ms
Wall time: 91 ms


In [52]:
model.inpu

KeyError: 'input_sub_x'

In [51]:
len(x["input_sub_x"])

4

In [6]:
collate_fn.feature_extractor.numerical_columns

['PPP2R5A',
 'ICAM2',
 'GSTA3',
 'NPL',
 'IZUMO1',
 'IL36G',
 'MAGED1',
 'PVALB',
 'CCL19',
 'CLSPN',
 'CPB1',
 'SKAP2',
 'DNPEP',
 'GIPC3',
 'IL10RA',
 'ATP2B4',
 'FOSB',
 'CXCL17',
 'NAP1L4',
 'CLNS1A',
 'CAPS',
 'CRHBP',
 'MASP1',
 'ERBB2',
 'DOK1',
 'CNTN2',
 'LTO1',
 'CRISP3',
 'LTB',
 'AK2',
 'CD80',
 'ASS1',
 'ALDH2',
 'AHNAK',
 'GPD1',
 'MME',
 'HLA-DRA',
 'RAB2B',
 'MLN',
 'PM20D1',
 'MORN4',
 'SERPINB1',
 'RBP5',
 'HBEGF',
 'TBCA',
 'GDNF',
 'NEDD4L',
 'DNAJA4',
 'KRT19',
 'KIAA1549',
 'PODXL2',
 'GPNMB',
 'SEPTIN3',
 'INPPL1',
 'DCTN2',
 'GUSB',
 'MRPL28',
 'WFIKKN2',
 'GSTP1',
 'AIDA',
 'SPINK1',
 'CSH1',
 'ADAM15',
 'NECTIN4',
 'M6PR',
 'NPTN',
 'IL3RA',
 'CD164L2',
 'MBL2',
 'LAMA4',
 'MEP1A',
 'KLK6',
 'ATP6V1D',
 'FKBP4',
 'AMBP',
 'ERBB4',
 'MRC1',
 'PON1',
 'PAPPA',
 'UPB1',
 'NFX1',
 'IL13RA2',
 'SOD2',
 'SPOCK1',
 'HIF1A',
 'CSRP3',
 'NCK2',
 'ROBO1',
 'PRKG1',
 'KRT8',
 'CLEC5A',
 'ARL2BP',
 'IFNGR1',
 'RNASE10',
 'ST13',
 'VSTM2L',
 'CEACAM20',
 'LRFN2',
 'CLEC14A

In [10]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    trainers,
)

tokenizer = Tokenizer(models.WordLevel())
# tokenizer.normalizer = normalizers.NFKC()
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# tokenizer.decoder = decoders.
trainer = trainers.WordLevelTrainer(
    vocab_size=3000,
    special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"],
    show_progress=True,
)

In [16]:
trainset[0].columns.tolist()

['C3',
 'KLK7',
 'GCHFR',
 'NHLRC3',
 'APOD',
 'GAPDH',
 'TP53I3',
 'CPA4',
 'ANXA2',
 'GRSF1',
 'IL25',
 'HMMR',
 'MRPL52',
 'PAIP2B',
 'THAP12',
 'FOS',
 'FGF9',
 'PITHD1',
 'THSD1',
 'PTGES2',
 'DEFB103A_DEFB103B',
 'ATP1B4',
 'CYB5A',
 'UNC79',
 'SLC34A3',
 'TAGLN3',
 'SLIRP',
 'CLASP1',
 'PSMC3',
 'KIR3DL2',
 'BEX3',
 'PFDN4',
 'BCL7A',
 'SMC3',
 'SLC28A1',
 'CDC123',
 'GJA8',
 'NMRK2',
 'GATA3',
 'CPLX2',
 'RASGRF1',
 'FGF7',
 'ANKRA2',
 'RBM25',
 'LYZL2',
 'CDK1',
 'CREB3',
 'CREBZF',
 'IGLON5',
 'SHC1',
 'ZP4',
 'TMOD4',
 'CEP152',
 'MYH7B',
 'CEP350',
 'CDC25A',
 'TRIM26',
 'MANEAL',
 'MUCL3',
 'GIMAP8',
 'CYTH3',
 'PDXDC1',
 'CLINT1',
 'MAPRE3',
 'EVI2B',
 'STAU1',
 'PCNA',
 'DNAJA1',
 'JMJD1C',
 'GAGE2A',
 'GAD1',
 'IZUMO1',
 'PDCL2',
 'PDE1C',
 'STOML2',
 'BSND',
 'MAPK13',
 'PDIA2',
 'BTLA',
 'MLLT1',
 'TPRKB',
 'ARHGAP5',
 'BTNL10',
 'PHLDB2',
 'PDIA5',
 'ATF4',
 'PRAME',
 'TOP1MT',
 'KHDC3L',
 'DCUN1D2',
 'IL3',
 'DCLRE1C',
 'ERCC1',
 'DCDC2C',
 'VCPKMT',
 'SPRING1',
 'M

In [17]:
tokenizer.train_from_iterator([trainset[0].columns.tolist()], trainer=trainer)
tokenizer.get_vocab()

{'PGLYRP2': 2029,
 'PLA2G1B': 2055,
 'SPINT1': 2535,
 'BMP6': 273,
 'DHODH': 806,
 'ENOPH1': 926,
 'ERBIN': 956,
 'ITPR1': 1468,
 'PPP2R5A': 2115,
 'ACSL1': 27,
 'PKD2': 2050,
 'MNDA': 1739,
 'ATXN3': 223,
 'C1S': 312,
 'MERTK': 1702,
 'NAGK': 1807,
 'TGFB1': 2652,
 'PLTP': 2071,
 'ERBB3': 954,
 'PLAT': 2059,
 'USP28': 2828,
 'POLR2F': 2092,
 'ITGB1BP2': 1456,
 'IGFBP4': 1343,
 'CCAR2': 376,
 'SERPINA9': 2396,
 'TNR': 2737,
 'LPA': 1598,
 'ITGB7': 1460,
 'NCK2': 1819,
 'NFKBIE': 1851,
 'WDR46': 2880,
 'GOLGA3': 1188,
 'TMED8': 2693,
 'F7': 985,
 'YWHAQ': 2900,
 'CCDC134': 377,
 'CCL25': 396,
 'ACAN': 16,
 'FAM171B': 997,
 'IGSF8': 1353,
 'NCR3LG1': 1822,
 'NID2': 1860,
 'SIGLEC1': 2438,
 'STOML2': 2576,
 'TANK': 2611,
 'GLYR1': 1177,
 'FZD10': 1099,
 'TDP1': 2634,
 'UXS1': 2831,
 'GH1': 1152,
 'DAPK2': 760,
 'KLF4': 1507,
 'PRCP': 2123,
 'SCLY': 2354,
 'ZNF830': 2910,
 'IFNGR1': 1327,
 'TMPRSS11B': 2698,
 'LY75': 1634,
 'BGLAP': 259,
 'CALB1': 344,
 'INPP5D': 1430,
 'GOPC': 1190,
 'LRI

In [18]:
tokenizer

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'save_pretrain'

In [8]:
from transformers import BertTokenizer, BertTokenizerFast

toknizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [4]:
toknizer.tra

AttributeError: can't set attribute 'vocab'

In [ ]:
toknizer.

In [59]:
o = toknizer(model.input_encoder.feature_extractor.numerical_columns)
o

{'input_ids': [[101, 1048, 12171, 2278, 24434, 2050, 2475, 102], [101, 1043, 20464, 2213, 102], [101, 1047, 13687, 17134, 102], [101, 26924, 2487, 102], [101, 1039, 2595, 20464, 2629, 102], [101, 8040, 2290, 2475, 102], [101, 1057, 2290, 16425, 102], [101, 9152, 2102, 2475, 102], [101, 3729, 2475, 9331, 102], [101, 2273, 2102, 102], [101, 1054, 5620, 10790, 102], [101, 7987, 2243, 2487, 102], [101, 1056, 20974, 2475, 102], [101, 11047, 2232, 2546, 2094, 2475, 102], [101, 11867, 2229, 2361, 2487, 102], [101, 2358, 5358, 2140, 2475, 102], [101, 8038, 2869, 2487, 102], [101, 6064, 3501, 2278, 2683, 102], [101, 8840, 16275, 2487, 102], [101, 27937, 7898, 2475, 102], [101, 3729, 2475, 102], [101, 14931, 8490, 2487, 2050, 1035, 14931, 8490, 2487, 2497, 102], [101, 24869, 11514, 2475, 102], [101, 14246, 2278, 2629, 102], [101, 16420, 25856, 27717, 102], [101, 18133, 2595, 2213, 2475, 102], [101, 6335, 11387, 2527, 102], [101, 21144, 2361, 2475, 102], [101, 21025, 2102, 2487, 102], [101, 17324

In [64]:
len(o["input_ids"])

2911

In [69]:
toknizer.cle

TypeError: PreTrainedTokenizerBase.clean_up_tokenization() missing 1 required positional argument: 'out_string'

In [68]:
model

TransTabForCL(
  (input_encoder): TransTabInputEncoder(
    (feature_processor): TransTabFeatureProcessor(
      (word_embedding): TransTabWordEmbedding(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (num_embedding): TransTabNumEmbedding(
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (align_layer): Linear(in_features=128, out_features=128, bias=False)
    )
  )
  (encoder): TransTabEncoder(
    (transformer_encoder): ModuleList(
      (0): TransTabTransformerLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
 

In [57]:
model.input_encoder.feature_extractor.numerical_columns

['LRRC37A2',
 'GCLM',
 'KLK13',
 'SPP1',
 'CXCL5',
 'SCG2',
 'UGDH',
 'NIT2',
 'CD2AP',
 'MENT',
 'RGS10',
 'BRK1',
 'TLR2',
 'MTHFD2',
 'SPESP1',
 'STOML2',
 'YARS1',
 'DNAJC9',
 'LONP1',
 'NPHS2',
 'CD2',
 'CTAG1A_CTAG1B',
 'BNIP2',
 'GPC5',
 'SFTPA1',
 'CPXM2',
 'IL20RA',
 'RBP2',
 'GIT1',
 'CBLN1',
 'SERPINB1',
 'ENOX2',
 'FCN1',
 'RTKN2',
 'RNF43',
 'FAM172A',
 'QSOX1',
 'CBX2',
 'IL17D',
 'LRP2',
 'TNFRSF4',
 'ATP1B2',
 'SH3BGRL2',
 'MCAM',
 'SNX18',
 'APOD',
 'PRG2',
 'EDDM3B',
 'GFER',
 'SEPTIN9',
 'DPY30',
 'CCL8',
 'KIAA2013',
 'ENOPH1',
 'FSTL3',
 'MANF',
 'ARMCX2',
 'CCL2',
 'RTN4R',
 'FST',
 'MGLL',
 'TRAF3IP2',
 'PSMC3',
 'LEO1',
 'CAT',
 'TIGIT',
 'NTRK3',
 'TNN',
 'CDH15',
 'BHLHE40',
 'GRP',
 'CARHSP1',
 'CRELD2',
 'LHPP',
 'SPINK2',
 'SIRT5',
 'NCAM1',
 'ARSA',
 'TERF1',
 'SIGLEC1',
 'DPP4',
 'IL6R',
 'MPO',
 'LCAT',
 'TOP1MT',
 'EDF1',
 'OFD1',
 'DPEP1',
 'CPB1',
 'NDRG1',
 'DNAJB2',
 'C5',
 'CSF1',
 'PILRA',
 'ADIPOQ',
 'ENSA',
 'USP28',
 'GATD3',
 'SLC4A1',
 'SAP18